## Langchain - LLMs: Get predictions from a language model

In [2]:
import os
from langchain.llms import OpenAI

open_ai_api = os.environ.get("OPENAI_API_KEY")
llm = OpenAI(temperature=0.9, openai_api_key=open_ai_api)

text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Colorful Socksation.


## Prompt Templates: Manage prompts for LLMs

> Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you are probably taking user input and constructing a prompt, and then sending that to the LLM.


In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


## Chains: Combine LLMs and prompts in multi-step workflows

>A chain in LangChain is made up of links, which can be either primitives like LLMs or other chains.

- The most core type of chain is an LLMChain, which consists of a PromptTemplate and an LLM.

- Extending the previous example, we can construct an LLMChain which takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM.

In [4]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

chain.run("colorful socks")

'\n\nFunky Feet Socks.'

## Agents: Dynamically Call Chains Based on User Input

> So far the chains we’ve looked at run in a predetermined order.

> Agents no longer do: they use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In [5]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0, openai_api_key=open_ai_api)


In [6]:
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [11]:
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What was the highest temperature in Patna yesterday in Celsius? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature and then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "highest temperature in Patna yesterday"
Observation: Patna Temperature Yesterday. Maximum temperature yesterday: 97 °F (at 3:30 pm) Minimum temperature yesterday: 77 °F (at 3:30 am) Average temperature ...
Thought: I need to convert the temperature from Fahrenheit to Celsius
Action: Calculator
Action Input: 97 °F to Celsius
Observation: Answer: 36.111111111111114
Thought: I now need to raise the temperature to the .023 power
Action: Calculator
Action Input: 36.111111111111114^.023
Observation: Answer: 1.0859897833978314
Thought: I now know the final answer
Final Answer: The highest temperature in Patna yesterday in Celsius was 36.111111111111114 and when raised to the .023 power it is 1.0859897833978314.

> Finished chain.


'The highest temperature in Patna yesterday in Celsius was 36.111111111111114 and when raised to the .023 power it is 1.0859897833978314.'

## Memory: Add State to Chains and Agents

- often, you may want a chain or agent to have some concept of “memory” so that it may remember information about its previous interactions.

In [12]:
from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. How can I help you today?


In [13]:
output = conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.
 That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
